## Pair Programming ANOVA





En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de 
normalización y estandarización.


Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una 
gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que 
hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.


📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, 
haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer 
cuando nos encontramos en esta situación.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Estadísticos
# ------------------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

In [4]:
df = pd.read_csv("../datos/sephora_website_dataset2.csv", index_col = 0)
df.sample(2)

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,marketingflags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer,rating_raiz
4269,2333037,Kérastase,Hair Masks,Resistance Mask for Damaged Hair Mini,no size,2.0,-0.315676,-0.363945,-0.616237,-0.623249,...,online only,no options,Which hair type is it good for?✔ Straight✔ Wav...,Suggested Usage:-Apply one or two walnut-sized...,unknown,1,0,0,0,-1.218083
5883,1401330,NARS,Foundation,Sheer Glow Foundation Pump,no size,3.0,-0.274131,-0.250646,-0.934287,-0.926547,...,0,no options,What it is:An optional pump for dispensing liq...,no instructions,unknown,0,0,0,0,-0.493753


> Definamos qué es un ANOVA.

Un ANOVA es un *análisis de varianza* que proporciona prueba de: 

- Las medias de varios *k* grupos son iguales
- Generaliza la prueba *t* a más de dos grupos

Es útil para:

- Comparar tres o más *medias* (grupos o variables) en busca de significación estadística.

Se deben cumplir *TRES ASUNCIONES* o supuestos:

1. Las muestras se seleccionan aleatoriamente de forma independiente, es decir, las medidas son independientes entre ellas.


2. Todos los grupos tienen distribuciones aproximadamente normales. Lo podemos comprobar mediante varias herramientas, visualmente o análiticamente. 


3. Las varianzas de los grupos tienen que ser iguales. 

---> En nuestro caso ya sabemos que nuestra VR (rating) no es normal, con lo que el ANOVA no resultará válido. Seguimos con el ejercicio para poder aprender la metodología y aplicarla en un futuro. 





### Empezamos nuestro ANOVA


La sintaxis, propia de R, es la siguiente:

> variable_respuesta ~ variable_predictora(s)

Se pueden añadir todas las variables predictoras que necesitemos. 
Dependiendo de qué efecto queramos usaremos una de las dos siguientes sintaxis:

1. Efectos no aditivos: 

```python
variable_respuesta ~ variable_predictora_1 + variable_predictora_2 + variable_predictora_3 ... + variable_predictora_n
``` 

1. Efectos aditivos: 

```python
variable_respuesta ~ variable_predictora_1 * variable_predictora_2 * variable_predictora_3 ... * variable_predictora_n
```

In [5]:
df.head(1)

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,marketingflags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer,rating_raiz
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,-0.312308,-0.311626,0.337912,0.468624,...,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1,0,0,0,0.116885


In [7]:
v_numericas = df.select_dtypes(include= ['int64', 'float64'] )
v_numericas

,id,rating,number_of_reviews,love,price,value_price,online_only,exclusive,limited_edition,limited_time_offer,rating_raiz
0,2218774,4.0,-0.312308,-0.311626,0.337912,0.468624,1,0,0,0,0.116885
1,2044816,4.5,-0.231463,-0.318715,0.337912,0.286645,1,0,0,0,0.393366
2,1417567,4.5,-0.287605,-0.321062,2.755091,2.591709,1,0,0,0,0.393366
3,1417617,4.5,-0.290974,-0.314020,1.482892,1.378518,1,0,0,0,0.393366
4,2218766,3.5,-0.314553,-0.359955,0.465132,0.569723,1,0,0,0,-0.177487
...,...,...,...,...,...,...,...,...,...,...,...
9163,2208502,2.0,-0.299956,-0.236563,-0.934287,-0.926547,0,1,1,0,-1.218083
9164,2298909,0.0,-0.316799,-0.375845,-0.743457,-0.501930,0,1,0,0,-4.440982
9165,2236750,0.0,-0.316799,-0.371644,-0.722254,-0.724348,0,1,0,0,-4.440982
9166,50,5.0,-0.265148,-0.382089,-0.001341,-0.036873,0,0,0,0,0.654869


In [7]:
df_numericas = v_numericas[['rating', 'number_of_reviews', 'love', 'price', 'value_price']]

In [8]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es rating
lm = ols('rating_raiz ~ number_of_reviews + love + price + value_price' , data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
number_of_reviews,1.0,58.343162,58.343162,59.576654,1.299457e-14
love,1.0,22.668085,22.668085,23.147334,1.524462e-06
price,1.0,0.601139,0.601139,0.613848,4.333629e-01
value_price,1.0,113.100981,113.100981,115.492164,8.863780e-27
Residual,9163.0,8973.286634,0.979296,NaN,NaN


> Analicemos los resultados:

- df: degrees of freedom
Grados de libertad: el número de observaciones en los datos que pueden variar libremente al estimar los parámetros estadísticos. 
En nuestro caso, al ser variables numéricas, solo pueden variar en un eje, el de los números reales -> por eso siempre es 1.

- sum_sq: Es una medida de variación o desviación de la media, calculando la suma de los cuadrados de las diferencias con respecto a la media.

En el ANOVA nos ayuda a expresar la variación total que puede atribuirse a diversos factores. 

La conversión de la suma de cuadrados en cuadrados medios dividiendo por los grados de libertad permite comparar proporciones y determinar *si hay diferencia significativa entre lo analizado*. 

Cuanto mayor proporción --> más afectan los tratamientos al resultado.

- mean_sq: división de la suma de cuadrados entre el número de grados de libertad

- F: test explicativo de la capacidad de los valores de la variable predictora para explicar una parte significativa de la *variación de la variable respuesta*.
-> En nuestro caso vemos que love tiene un valor F de 23.147334

- PR(>F): referente al pvalor. 
-> Si el pvalor es <= 0.05 debemos rechazar la hipótesis nula y concluimos que la variable SÍ influye en el rating
-> Si el valor es >= 0 no contamos con suficientes evidencias para rechazar la hipótesis y no tiene influencia sobre el rating. 
----> En nuestro caso el PR(>F) es mayor que 0 así que love no tiene influencia sobre rating. 



El método .summary nos resume la recta que ajusta nuestro modelo

In [9]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            rating_raiz   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     49.71
Date:                Sun, 21 May 2023   Prob (F-statistic):           1.90e-41
Time:                        14:58:51   Log-Likelihood:                -12910.
No. Observations:                9168   AIC:                         2.583e+04
Df Residuals:                    9163   BIC:                         2.587e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          2.385e-16      0.010   2.31e-14      1.000      -0.020       0.020
number_of_reviews     0.0198      0.016      1.271      0.204      -0.011       0.050
love                  0.0762      0.016      4.906      0.000       0.046       0.107
price                 0.6094      0.057     10.709      0.000       0.498       0.721
value_price          -0.6117      0.057    -10.747      0.000      -0.723      -0.500
==============================================================================
Omnibus:                     6851.053   Durbin-Watson:                   1.751
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            89489.914
Skew:                          -3.666   Prob(JB):                         0.00
Kurtosis:                      16.435   Cond. No.                         11.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Analizamos los resultados de summary:

- R-squared: tiene que tener un valor entre 0 y 1, en el que 0 no explica la variabilidad de la variable respuesta. 

->En nuestro caso 0.021, indica que el modelo no explica toda la variabilidad de la variable respuesta 

- Pvalor: de 0, como anteriormente

- El coef: de 0.0762 -> por cada unidad que cambia love el rating aumenta un 7.662%
